In [5]:
import re
import pandas as pd

In [64]:
df = pd.read_csv('data/soal1.csv')
df.columns = [i.lower() for i in df.columns]
df.head()

,log_id,guest_id,check_in_date,duration_string,location_id,guest_type
0,1001,G1234,2024-03-15,"2 days, 10 hours",LOC01,New_User
1,1002,G5678,2023-12-28,"1 day, 5 hours",LOC03,returning
2,1003,G9012,2024-06-01,"4 days, 0 hours",LOC01,Returning Guest
3,1004,G3456,2024-01-05,10 hours,LOC02,new
4,1005,G7890,2024-07-20,"5 days, 15 hours",LOC04,Returning


In [65]:
df.dtypes

log_id              int64
guest_id           object
check_in_date      object
duration_string    object
location_id        object
guest_type         object
dtype: object

# Calculate Stay_Duration_Hours

In [54]:
def clean_duration(row):
    result = 0
    if re.search('week(?:s)*', row, flags=re.I):
        temp = re.findall("(\d+)\s*(?=week(?:s)*)", row, flags=re.I)[0]
        result += float(temp) * 7 * 24
    if re.search('day(?:s)*', row, flags=re.I):
        temp = re.findall("(\d+)\s*(?=day(?:s)*)", row, flags=re.I)[0]
        result += float(temp) * 24
    if re.search('hour(?:s)*', row, flags=re.I):
        temp = re.findall("(\d+)\s*(?=hour(?:s)*)", row, flags=re.I)[0]
        result += float(temp)
    return result

In [55]:
df['duration_string'] = df['duration_string'].apply(clean_duration)

In [56]:
df.head()

,log_id,guest_id,check_in_date,duration_string,location_id,guest_type
0,1001,G1234,2024-03-15,58.0,LOC01,New_User
1,1002,G5678,2023-12-28,29.0,LOC03,returning
2,1003,G9012,2024-06-01,96.0,LOC01,Returning Guest
3,1004,G3456,2024-01-05,10.0,LOC02,new
4,1005,G7890,2024-07-20,135.0,LOC04,Returning


# Standardize Guest_Type

In [57]:
def clean_guest_type(row):
    if re.search('new|first[\s-]time|baru', row, flags=re.I):
        return 'New'
    if re.search('returning|repeat|kembali', row, flags=re.I):
        return 'Returning'
    return row

In [58]:
set(df['guest_type'].apply(clean_guest_type)) == {'New', 'Returning'}

True

In [59]:
df['guest_type'] = df['guest_type'].apply(clean_guest_type)

# Filter Data

In [71]:
df['check_in_date_temp'] = pd.to_datetime(df['check_in_date'], errors='coerce')

In [73]:
df[df['check_in_date_temp'].isna()]

,log_id,guest_id,check_in_date,duration_string,location_id,guest_type,check_in_date_temp
46,1047,G7046,2024-16-05,"4 days, 1 hour",LOC01,New_User,NaT


In [74]:
# asumsi typo 2024-16-05 -> 2024-05-16

In [82]:
df.loc[df['check_in_date'] == '2024-16-05', 'check_in_date'] = '2024-05-16'

In [83]:
df['check_in_date_temp'] = pd.to_datetime(df['check_in_date'], errors='coerce')
df[df['check_in_date_temp'].isna()]

,log_id,guest_id,check_in_date,duration_string,location_id,guest_type,check_in_date_temp


In [84]:
df['check_in_date'] = pd.to_datetime(df['check_in_date'], errors='coerce')
df.drop(columns='check_in_date_temp', inplace=True)

In [91]:
df = df[df['check_in_date'] >= '2024-01-01']
df.head()

,log_id,guest_id,check_in_date,duration_string,location_id,guest_type
0,1001,G1234,2024-03-15,"2 days, 10 hours",LOC01,New_User
2,1003,G9012,2024-06-01,"4 days, 0 hours",LOC01,Returning Guest
3,1004,G3456,2024-01-05,10 hours,LOC02,new
4,1005,G7890,2024-07-20,"5 days, 15 hours",LOC04,Returning
5,1006,G1122,2024-02-14,"1 day, 2 hours",LOC05,Repeat
